# Fasam - NLP Competition

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

['df_train.csv', 'df_valid.csv']


Esse notebook descreve o pipelie para utilização na competição do **Kaggle da Fasam**. Faz parte da avaliação prática dos alunos da turma de Deep Learning.


### Roteiro do Notebook

* Leitura do Dataset
* Criação do Modelo
* Avaliação e Criação do arquivo de submission.csv

### Problema


Uma revista precisa catalogar todas as suas notícias em diferentes categorias. O objetivo desta competição é desenvolver o melhor modelo de aprendizagem profunda para prever a categoria de novas notícias.


<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Untitled-Diagram.png
" style="width: 400px;"/>


As categorias possíveis são:

* ambiente
* equilibrioesaude
* sobretudo
* educacao
* ciencia
* tec
* turismo
* empreendedorsocial
* comida


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

sns.set(style="ticks")
warnings.filterwarnings("ignore")

In [3]:
# Bibliotecas do keras
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras import regularizers
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.optimizers import Adam

Using TensorFlow backend.


## Leitura do dataset de treinamento

In [4]:

#!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/contributed/pt/elmo_pt_weights.hdf5
# Leitura do Dataset
df = pd.read_csv('../input/df_train.csv')
df['title_length'] = df['title'].str.len()
df['text_length'] = df['text'].str.len()
print(df.shape)
df.head(1000)


(7894, 5)


,title,text,category,title_length,text_length
0,Casa da Barra Funda tem clima roceiro e receit...,"No Quem Quer Pão 75, casa de clima roceiro em ...",comida,58,2019
1,Professores de SP decidem manter greve; grupo ...,Professores da rede estadual decidiram manter ...,educacao,73,2338
2,"Em segunda edição, concurso paga R$ 35 mil par...",O Prêmio FedEx para Pequenas Empresas está com...,empreendedorsocial,65,1235
3,Usar maconha por anos não faz tão mal para a s...,No esforço de entender os efeitos da maconha s...,equilibrioesaude,69,4312
4,Baleia-azul percorre 5.200 km e revela a cient...,"Isabela, uma baleia-azul, ajudou os cientistas...",ciencia,69,4106
5,Líderes inovadores se reunem em rede para comp...,A Rede Folha de Empreendedores Socioambientais...,empreendedorsocial,72,2043
6,"Conheça Pandora, atração milionária da Disney ...",Luzes. Câmera. Ação. Há menos de duas semanas...,turismo,67,5291
7,Fiesp organiza quarta edição de maratona hacker,Atenção hackers e programadores! É hora de cor...,empreendedorsocial,47,1367
8,Praia do Forte mistura natureza com resorts es...,Localizada a cerca de 50 quilômetros da capita...,turismo,65,5886
9,App de agência avisa quando visto e passaporte...,App de agência avisa quando visto e passaporte...,turismo,68,3070


In [5]:
df.max()

title            Útero doado oferece uma nova chance de dar à luz
text            – Vamos beber umas hoje à noite?  Eta danado! ...
category                                                  turismo
title_length                                                  116
text_length                                                 32120
dtype: object

Todos os artigos contêm o **título, descrição e link** da matéria original. Por último a categoria que pertence esse artigo.

In [22]:
def preprocess(dataframe):
    print("In preprocess function.")

    dataframe1 = remove_nan(dataframe)
    dataframe2 = lowercase(dataframe1)
    dataframe3 = remove_whitespace(dataframe2)

    # Remove emails and websites before removing special characters
    dataframe4 = remove_emails(dataframe3)
    dataframe5 = remove_website_links(dataframe4)

    dataframe6 = remove_special_characters(dataframe5)
    dataframe7 - remove_numbers(dataframe6)
    remove_stop_words(dataframe8) # Doesn't return anything for now
    dataframe7 = tokenize(dataframe6)

    print(f"Sample of preprocessed data: {dataframe4.head()}")

    return dataframe7

def remove_nan(dataframe):
    """Pass in a dataframe to remove NAN from those columns."""
    return dataframe.dropna()

def lowercase(dataframe):
    print("Converting dataframe to lowercase")
    #lowercase_dataframe = dataframe.apply(lambda x: x.str.lower())    
    lowercase_dataframe = dataframe.applymap(lambda s:s.lower() if type(s) == str else s)   
    return lowercase_dataframe


def remove_special_characters(dataframe):
    print("Removing special characters from dataframe")
    no_special_characters = dataframe.replace(r'[^A-Za-z0-9 ]+', '', regex=True)
    return no_special_characters

def remove_numbers(dataframe):
    print("Removing numbers from dataframe")
    removed_numbers = dataframe.str.replace(r'\d+','')
    return removed_numbers

def remove_whitespace(dataframe):
    print("Removing whitespace from dataframe")
    # replace more than 1 space with 1 space
    merged_spaces = dataframe.text.replace(r"\s\s+",' ')
    # delete beginning and trailing spaces
    trimmed_spaces = merged_spaces.apply(lambda x: x.text.strip())
    return trimmed_spaces

def remove_stop_words(dataframe):
    # TODO: An option to pass in a custom list of stopwords would be cool.
    set(stopwords.words('portuguese'))

def remove_website_links(dataframe):
    print("Removing website links from dataframe")
    no_website_links = dataframe.text.replace(r"http\S+", "")
    return no_website_links

def tokenize(dataframe):
    tokenized_dataframe = dataframe.apply(lambda row: word_tokenize(row))
    return tokenized_dataframe

def remove_emails(dataframe):
    no_emails = dataframe.text.replace(r"\S*@\S*\s?")
    return no_emails

def expand_contractions(dataframe):
    # TODO: Not a priority right now. Come back to it later.
    return dataframe

In [23]:
df = preprocess(df)

In preprocess function.
Converting dataframe to lowercase
Removing whitespace from dataframe


AttributeError: 'str' object has no attribute 'text'

In [ ]:
## Definição de alguns parâmetros dos modelos e tokenização

# Tamanho da sequencia
#seq_size     = 64
seq_size     = 116

# Máximo de tokens 
max_tokens   = 2500
#max_tokens   = 5500

# Tamanho do embedding
#embed_dim    = 128
#embed_dim    = 256
#embed_dim    = 512
embed_dim    = 256

Iremos utilizar o titulo para o nosso modelo baseline. O processo abaixo cria o **input** da nossa rede e prepara o **target**

In [ ]:
## Utilizaremos apenas o .title (input) e o .category (target) da nossa rede
# Textos
#text         = df['title'].values
text         = df['text'].values
tokenizer    = Tokenizer(num_words=max_tokens, split=' ')

# Transforma o texto em números
tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)  

# Cria sequencias de tamanho fixo (input: X)
X = pad_sequences(X, maxlen=seq_size)

In [ ]:
# Categoriza o target "category" -> [0,..., 1] (output: y)
Y_classes = pd.get_dummies(df['category']).columns
Y         = pd.get_dummies(df['category']).values

In [ ]:
(X.shape, Y.shape)

## Criação do Modelo

Iremos utilizar uma RNN em um modelo simples.

In [ ]:
def base_model():
    model = Sequential()
    
    # Embedding Layer
    model.add(Embedding(max_tokens, embed_dim, 
                        input_length = seq_size))
    # RNN Layer
    #model.add(LSTM(seq_size))
   
    model.add(LSTM(seq_size, activation='tanh', recurrent_activation='hard_sigmoid', 
                      use_bias=True, kernel_initializer='glorot_uniform', 
                      recurrent_initializer='orthogonal', 
                      bias_initializer='zeros', 
                      unit_forget_bias=True, 
                      kernel_regularizer=None, 
                      recurrent_regularizer=None, 
                      #kernel_regularizer=regularizers.l1(0.1), 
                      #recurrent_regularizer=regularizers.l2(0.1), 
                      bias_regularizer=None, 
                      activity_regularizer=None, 
                      kernel_constraint=None, 
                      recurrent_constraint=None, 
                      bias_constraint=None, dropout=0.0, 
                      recurrent_dropout=0.0, implementation=1, 
                      return_sequences=False, 
                      return_state=False, 
                      go_backwards=False, 
                      stateful=False, 
                   unroll=False))
   
    
    '''
    model.add(GRU(seq_size, activation='tanh', 
                     recurrent_activation='hard_sigmoid', 
                     use_bias=True, kernel_initializer='glorot_uniform', 
                     recurrent_initializer='orthogonal', 
                     bias_initializer='zeros', 
                     kernel_regularizer=None,                      
                     recurrent_regularizer=None, 
                     bias_regularizer=None, 
                     activity_regularizer=None, 
                     kernel_constraint=None, 
                     recurrent_constraint=None, 
                     bias_constraint=None, dropout=0.0, 
                     recurrent_dropout=0.0, 
                     implementation=1, 
                     return_sequences=False, 
                     return_state=False, 
                     go_backwards=False, 
                     stateful=False, 
                     unroll=False, 
                     reset_after=False))
     '''
    
    # Dense Layer
    model.add(Dense(len(Y_classes), activation='softmax'))
    #model.add(Dense(len(Y_classes), activation='relu'))
    
    #otim = SGD(lr=1e-5, decay=1e-6, momentum=0.9, nesterov=True)
    #otim = SGD(lr=0.5, decay=0.0, momentum=0.9, nesterov=True)
    otim = Adam(lr=0.01, decay=0.0)
    
    
    model.compile(loss = 'categorical_crossentropy', 
                  #optimizer='adam',
                  optimizer=otim,
                  metrics = ['accuracy'])
    
    model.summary()
    
    return model

base_model = base_model()


In [ ]:
# Separa o dataset em dados de treinamento/teste
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, 
                                                      test_size = 0.5, 
                                                      random_state = 42)

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#filepath="weights-betters.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

#base_model.load_weights('weights-improvement-04-0.86.hdf5')
#base_model.load_weights('weights-improvement-10-0.78.hdf5')

# Treina o modelo
hist = base_model.fit(X_train, Y_train, 
              validation_data =(X_valid, Y_valid),
              batch_size=8, nb_epoch = 5, callbacks=callbacks_list, verbose = 1)

## Avaliação e Criação do arquivo de submission.csv

In [ ]:
# Avaliação do modelo para o dataset de test

# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

val_loss, val_acc = base_model.evaluate(X_valid, Y_valid)

print('A acurácia do modelo está de: '+str(val_acc*100)+'%')

### Criando arquivo de submission para o Kaggle

In [ ]:
# Leitura do Dataset de validação dos resultados
df_valid = pd.read_csv('../input/df_valid.csv')
print(df_valid.shape)
df_valid.head(100)


O dataset de validação, o que será utilizado para calcular o Ranking no Kaggle, contêm apenas as informações de Título e Texto do arquivo.  O modelo criado deve ser capaz de classificar em qual das categorias esse artigo pertence

In [ ]:
def predict(text):
    '''
    Utiliza o modelo treinado para realizar a predição
    '''
    new_text = tokenizer.texts_to_sequences(text)
    new_text = pad_sequences(new_text, maxlen=seq_size)
    pred     = base_model.predict_classes(new_text)#[0]
    return pred

In [ ]:
# Como utilizamos o titulo no treinamento, iremos utilizar o titulo na predição também

#pred         = predict(df_valid.title)
pred         = predict(df_valid.text)
pred_classes = [Y_classes[c] for c in pred]
pred_classes[:5]

In [ ]:
# Atualizando a categoria dos artigos no dataset de validação
df_valid['category'] = pred_classes
df_valid.head(2000)

In [ ]:
def create_submission(df):
    f = open('submission_valid.csv', 'w')
    f.write('id,category\n')
    for i, row in df.iterrows():
        f.write('{},{}\n'.format(i, row.category))
    f.close()
    
# Criando o arquivo submission_valid.csv contendo os dados para cálculo do ranking no kaggle
# Esse arquivo deve ser enviado para o kaggle
create_submission(df_valid)    